In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import os
import re
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import Counter
import time
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

In [37]:
def generate_tuple(row):
    list_of_tuples = []
    for item in row:
        for string in item.split():
            if string.isnumeric():
                # Take everything up to the number as a single string
                list_of_tuples.append((item[:item.find(string) - 1], string)) # Create a tuple (Topic, Frequency)
    return list_of_tuples

In [38]:
def a(test_str):
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '(':
            skip1c += 1
        elif i == ')' and skip1c > 0:
            skip1c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

In [39]:
path = '../../Data/CorporaData'

In [40]:
files = []
for folder in os.listdir(path):
    if folder != '.DS_Store' and folder != '.ipynb_checkpoints':
        for file in os.listdir(os.path.join(path, folder)):
            if file != '.ipynb_checkpoints':
                files.append(('../../Data/CommercialData/{}/{}_{}.txt'.format(
                    file.split('.')[0], file.split('.')[0], file.split('.')[2]), os.path.join(path, os.path.join(folder, file))))

In [41]:
for tpl in files:
    df = pd.read_csv(tpl[1])
    df.columns = ['URL', 'Title', 'Text']
    df['Text'] = df['Text'].apply(lambda text: str(text))
    df['Text'] = df['Text'].apply(lambda text: text.replace('[[TIME.START]] ', '('))
    df['Text'] = df['Text'].apply(lambda text: text.replace(' [[TIME.END]]', ')'))
    df['Text'] = df['Text'].apply(lambda text: str(text).split('TOPICS: TOPIC FREQUENCY ')[0])
    df['Text'] = df['Text'].apply(lambda text: str(text)[str(text).find('[[TITLE.END]] ') + len('[[TITLE.END]] '):])
    df['Text'] = df['Text'].apply(lambda text: re.sub('<[^>]+>', '', text))
    df['Text'] = df['Text'].apply(lambda text: re.sub(r'[^\w\s]', '', text))
    df['Text'] = df['Text'].apply(lambda text: ' '.join([word for word in text.split() if word not in cachedStopWords]))
    df['Text'] = df['Text'].apply(lambda text: a(text))
    corpora_df.to_csv('../../OutputCommercialsDataframes/{}/{}_{}_{}'.format(tpl[1].split('/')[4].split('.')[0], tpl[1].split('/')[4].split('.')[0], tpl[1].split('/')[4].split('.')[2], 'output.csv'), index=False)

In [21]:
comm_path = '../../OutputCommercialsDataframes'

In [22]:
files = []
for folder in os.listdir(comm_path):
    if folder != '.DS_Store' and folder != '.ipynb_checkpoints':
        for file in os.listdir(os.path.join(comm_path, folder)):
            if file != '.ipynb_checkpoints':
                files.append(('../../Data/CommercialData/{}/{}_{}.txt'.format(
                    file.split('_')[0], file.split('_')[0], file.split('_')[1]), os.path.join(comm_path, os.path.join(folder, file))))

In [23]:
files[0]

('../Data/CommercialData/FOXNEWS/FOXNEWS_2016.txt',
 '../OutputCommercialsDataframes/FOXNEWS/FOXNEWS_2016_output.csv')

In [24]:
commercials = {'Bloomberg': {}, 
              'CNBC': {},
              'CNN': {},
              'FBC': {},
              'FOXNEWS': {},
              'MSNBC': {}}

In [25]:
for tpl in files:
    corpora_df = pd.read_csv(tpl[1])
    
    with open(tpl[0], 'r') as f: # Get the potential commericials
        comm_lines = f.readlines()
        f.close()
    
    
    for i in range(len(corpora_df.index)):
        commercials[tpl[1].split('/')[2]][corpora_df.loc[i, 'Title']] = []
        
    for i in range(len(corpora_df.index)):
        segmented_text = str(corpora_df.loc[i, 'RawText_preprocessed']).split('[[TIME.START]]')
        for segment in segmented_text:
            for comm in comm_lines:
                if comm in segment:
                    commercials[tpl[1].split('/')[2]][corpora_df.loc[i, 'Title']].append(segment)
    break

In [29]:
for k,v in commercials['FOXNEWS'].items():
    if v:
        print(k)